In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Multi genbank and stacking



In [ ]:
import genomenotebook as gn
import os
from bokeh.io import output_notebook #|hide_line

## Plotting a linked stack of contigs

`colored_genbank.gb` is a genbank file with four identical contigs. Plotting the contigs as a GenomeStack links the x-axes.

In [ ]:
output_notebook(hide_banner=True) #|hide_line
data_path = gn.get_example_data_dir()
gb_path = os.path.join(data_path, "colored_genbank.gb")
g = gn.GenomeStack.from_genbank(gb_path,
                                width=700, 
                                show_seq=True, 
                                search=False, 
                                feature_types=["CDS", "Domainator"], 
                                color_attribute="Color", 
                                label_angle=0, 
                                show_labels=True, 
                                feature_height=0.15, 
                                label_vertical_offset=-0.15, 
                                label_justify="left", 
                                label_horizontal_offset = 5, 
                                glyphs=gn.get_default_glyphs(box_colors=("Cyan",)), 
                                feature_name={"CDS":"gene_id","Domainator":"name"},
                                z_stack=True) 
g.show()

## Additional examples of plotting stacks of contigs

In [ ]:
output_notebook(hide_banner=True) #|hide_line
data_path = gn.get_example_data_dir()
gb_path = os.path.join(data_path, "MT_nbs.gb")
g = gn.GenomeStack.from_genbank(gb_path,
                                width=500, 
                                show_seq=True, 
                                search=False, 
                                feature_types=["source", "CDS", "Domainator", "Domain_Search"], 
                                color_attribute="Color", 
                                label_angle=0, 
                                show_labels=True, 
                                feature_height=0.15, 
                                label_vertical_offset=-0.15, 
                                label_justify="left", 
                                label_horizontal_offset = 5, 
                                glyphs=gn.get_default_glyphs(arrow_colors=("grey",), box_colors=("Cyan",)), 
                                feature_name={"CDS":"gene_id","Domainator":"name", "Domain_Search":"name"},
                                z_stack=True) #, "Domainator", "Domain_Search"
g.show()

In [ ]:
#| hide
output_notebook(hide_banner=True) #|hide_line
data_path = gn.get_example_data_dir()
gb_path = os.path.join(data_path, "hmf_pathway_variants.gbk")
g = gn.GenomeStack.from_genbank(gb_path,
                                width=700, 
                                show_seq=True, 
                                search=False, 
                                feature_types=["CDS", "motif", "overhang", "RBS", "ribozyme", "spacer", "Design_Complexit"], 
                                #color_attribute="Color", 
                                label_angle=0, 
                                show_labels=False, 
                                feature_height=0.15, 
                                label_vertical_offset=-0.15, 
                                label_justify="left",
                                label_horizontal_offset = 5, 
                                glyphs=gn.get_default_glyphs(arrow_colors=("grey",), box_colors=("Cyan",)), 
                                feature_name="label",
                                z_stack=True) 

g.show()